In [1]:
from Crypto.Cipher import Salsa20
from scipy import stats
import random
from numpy.random import Generator
from randomgen import HC128
from pytrivium import Trivium

def bitstring_to_bytes(s):
    v = int("1"+s, 2)
    b = bytearray()
    while v:
        b.append(v & 0xff)
        v >>= 8
    b.pop()
    return bytes(b[::-1])

def byte_xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def generate_binary_string(n):
    return ''.join(str(random.randint(0, 1)) for _ in range(n))

In [2]:
key_length=128
# The plaintext should consist of pure 0s to retrieve the keystream
plaintext = bitstring_to_bytes("00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000")
# We want to keep the nonce constant so that no additional paramethers are introduced
nonce_string = "0000000000000000000000000000000000000000000000000000000000000000"
nonce = bitstring_to_bytes(nonce_string)

In [3]:
def compute_higher_order_derivative(cipher_name = "salsa", order=20):
    starting_string=generate_binary_string(key_length-order)
    sum_string=bitstring_to_bytes("".zfill(key_length))
    for i in range(2**order):
        binary_ending = f'{i:0{order}b}'
        secret = bitstring_to_bytes(starting_string+binary_ending)
        if cipher_name == "salsa":
            cipher = Salsa20.new(key=secret,nonce=nonce)
            msg =  cipher.encrypt(plaintext)
        elif cipher_name == "hc128":
            # The key is a 256-bit integer that contains both the key (lower 128 bits) and initial values (upper 128-bits) for the HC-128 cipher
            seed = nonce_string+nonce_string
            hc_key = int(seed+starting_string+binary_ending, 2)
            hc128 = HC128(key=hc_key)
            gen = Generator(hc128)
            msg = gen.bytes(16)
        elif cipher_name == "trivium":
            trivium = Trivium()
            iv = [0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00,0x00]
            # Key length for trivium is 80 bits so we skip the first 6 bytes
            key=secret[6:]
            trivium.initialize(key,iv)
            trivium.update(16)
            output = trivium.finalize()
            pad=""
            for i in output:
                pad += "{:08b}".format(i)
            msg = bitstring_to_bytes(pad)
        else:
            raise ValueError("not a supported cipher")
        sum_string = byte_xor(sum_string, msg)
    #print(sum_string)
    binary_string = "{:08b}".format(int(sum_string.hex(),16))
    #print(binary_string)
    #print("hamming weight is: "+str(binary_string.count('1')/len(binary_string)))
    
    return binary_string

In [5]:
results = []
# Number of samples for chi-squared test is recommended to be greater than 13
for i in range(20):
    results.append(compute_higher_order_derivative(cipher_name="trivium",order = 25))
hamming_weights=map(lambda x:x.count('1'),results)
stats.chisquare(list(hamming_weights))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Power_divergenceResult(statistic=9.830721003134796, pvalue=0.956998601706501)

In [7]:
results = []
# Number of samples for chi-squared test is recommended to be greater than 13
for i in range(20):
    results.append(compute_higher_order_derivative(cipher_name="salsa",order = 25))
hamming_weights=map(lambda x:x.count('1'),results)
stats.chisquare(list(hamming_weights))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Power_divergenceResult(statistic=10.139904610492849, pvalue=0.9494097592189479)

In [9]:
results = []
# Number of samples for chi-squared test is recommended to be greater than 13
for i in range(20):
    results.append(compute_higher_order_derivative(cipher_name="hc128",order = 25))
hamming_weights=map(lambda x:x.count('1'),results)
stats.chisquare(list(hamming_weights))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


Power_divergenceResult(statistic=9.87692307692308, pvalue=0.9559167318711057)